## Grepping functions from the vulnerability context of the file.

# Merging output generated by FlawFinder and CppCheck

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import lizard
import subprocess as sub
from pylibsrcml import srcml
import os
import xml.etree.ElementTree as et 
import sys
import csv
# from lxml import etree
import pandas as pd
import subprocess as sub
import sys
from io import BytesIO, StringIO
import numpy as np
import itertools

# add parent dir to access modules
sys.path.append('../')

from src.tools import apply_flawfinder, xml2df, apply_cppcheck
from src.scanner import  merge_flawfinder_cppcheck, project_flaws


# # CppCheck scaning a dir
chk_dir = '../data/projects/contiki-2.4/apps/'

df_ff = apply_flawfinder(chk_dir)
# df_cc = apply_cppcheck(chk_dir)
# df_flaw = merge_flawfinder_cppcheck(df_ff, df_cc)
# df_flaw
df_ff

,File,Line,Column,DefaultLevel,Level,Category,Name,Warning,Suggestion,Note,CWEs,Context,Fingerprint,ToolVersion,RuleId,HelpUri
0,../data/projects/contiki-2.4/apps/ping6/ping6.c,92,5,4,4,buffer,scanf,"The scanf() family's %s operation, without a l...","Specify a limit to %s, or use a different inpu...",NaN,"CWE-120, CWE-20","scanf(""%s"", command);",afbbfbe50ed79090490df46f9686822738867ed6bfab6c...,2.0.19,FF1020,https://cwe.mitre.org/data/definitions/120.html
1,../data/projects/contiki-2.4/apps/ping6/ping6.c,99,8,4,0,buffer,scanf,"The scanf() family's %s operation, without a l...","Specify a limit to %s, or use a different inpu...",No risky scanf format detected.,"CWE-120, CWE-20","if(scanf("" %04x:%04x:%04x:%04x:%04x:%04x:%...",faa93e217704a4dc4599087053a7b1e53fd9b33ef2dc8d...,2.0.19,FF1020,https://cwe.mitre.org/data/definitions/120.html


In [2]:
df_fun  = project_flaws(df_flaw)
df_fun 

NameError: name 'df_flaw' is not defined

## The Rough Auditing Tool for Security is an open source tool developed by Secure Software Engineers
https://security.web.cern.ch/recommendations/en/codetools/rats.shtml \
Example: \
`rats --quiet --xml -w 3 data/projects/contiki-2.4/apps/`

In [182]:
def xml2df(xml):
    """convert xml file of rats tool to dataframe"""
    if isinstance(xml, str):
        xtree = et.fromstring(xml)
        df = pd.DataFrame()
        
        for err in xtree.findall("vulnerability"):
            dt = {'severity': err.find('severity').text, 
                        'type': err.find('type').text if err.find('type')!=None else None,
                        'message': err.find('message').text
                        } 
            for loc in err.findall("file"):
                dt['file'] = loc.find('name').text

                for line in loc.findall('line'):
                    dt['line'] = line.text
                    df = pd.concat([df, pd.DataFrame([dt])], 
                                ignore_index=True)
            # df = df.drop(columns=["file"], axis=1)
        return df
    else:
        return None

def apply_rats(file_or_dir, xmlfile="output.xml"):
    """find flaws in the file using CppCheck tool
    example commands:
    !cppcheck --template=gcc ../data/projects/contiki-2.4/apps/ 2> err.txt
    !cppcheck --template="{file}; {line}; {severity}; {message}; {code}"
    --template-location=" {file};{line}; {info};{code}\n" ../data/projects/contiki-2.4/apps/ 2> err.txt
    """
    # rats --quiet --xml -w 3 <dir_or_file>
    cmd = ["rats --quiet --xml -w 3 " + file_or_dir]
    process = sub.Popen(cmd, shell=True, stdout=sub.PIPE)
    output = process.stdout.read().decode("utf-8")
    # TODO: try not to create output.xml file instead use BytesIO.
    df = xml2df(output)
    # df["line"] = df.line.astype(int)
    return df

chk_dir = "../data/projects/contiki-2.4/apps/"
df_flaw = apply_rats(chk_dir)
df_flaw.to_csv('rats.csv')
df_flaw.head(20)

,severity,type,message,file,line
0,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//program-han...,187
1,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//ftp/ftpc.c,57
2,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//ftp/ftpc.c,99
3,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//ftp/ftpc.c,104
4,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//ftp/ftpc.c,111
5,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//twitter/twi...,57
6,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//twitter/twi...,58
7,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//twitter/twi...,59
8,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//shell/shell...,62
9,High,fixed size global buffer,\n Extra care should be taken to ensure tha...,../data/projects/contiki-2.4/apps//shell/shell...,101
